# NYC Event Explorer
Team 3: Xuke Chen, Inna Diambekova, Shah Alam, Asher Vaysblat

### **Import Dataset**

In [41]:
#import libraries needed
import requests
import pandas as pd
from datetime import datetime, date

In [42]:
#Fetch the data in JSON format
url="https://data.cityofnewyork.us/resource/tvpp-9vvx.json?$limit=7243"
results = requests.get(url).json()

#Creating a DataFrame from JSON data
df=pd.DataFrame(results)
#Filter out unrelated events
df = df[~df['event_name'].str.contains('Lawn|Landscape|Miscellaneous', case=False)]
# Create date and time columns
df['start_date'] = df['start_date_time'].str[:10]
df['start_time'] = df['start_date_time'].str[-12:]
df['start_time'] = df['start_time'].str[:8]
df['end_date'] = df['end_date_time'].str[:10]
df['end_time'] = df['end_date_time'].str[-12:]
#Convert 24hr time to 12hr time
def convert_to_12hr(time_24hr):
    time_obj = datetime.strptime(time_24hr, "%H:%M:%S")
    return time_obj.strftime("%I:%M:%S %p")
df['start_time'] = df['start_time'].apply(convert_to_12hr)
#Filter only special event
df = df[df['event_type'] == 'Special Event']
#Create a day column
for d in df.index:
  df.at[d,'start_day']=datetime.strptime(df.at[d,'start_date'], '%Y-%m-%d').date().strftime('%A')
  df.at[d,'end_day']=datetime.strptime(df.at[d,'end_date'], '%Y-%m-%d').date().strftime('%A')

# Group by 'EventName', aggregate with the earliest 'StartDateTime' and latest 'EndDateTime' for repeating events
consolidated_df = df.groupby('event_name').agg({
    'start_date': 'min',
    'end_date': 'max',
    'start_time' : 'min',
    'start_day' : 'min',
    'end_time' :'max',
    'end_day' : 'max'
}).reset_index()
# Merge the consolidated dates back to the original DataFrame
final_df = pd.merge(df[['event_name','event_borough','event_location']], consolidated_df, on='event_name', how='left')
# Drop duplicates to keep only unique event names
final_df.drop_duplicates(subset='event_name', keep='first', inplace=True)
# re-index
df = final_df.reset_index(drop=True)

#Reorder columns and delete unnecessary columns
df=df[['event_name','start_date','start_time','start_day','end_date','end_time','end_day','event_borough','event_location']]
#pick our editor's choice
editor= df[df['event_name'].isin(['2023 LIC Holiday Tree and Lighting in the Vernon Mall','Big Apple Circus','2023 HOLIDAY MARKET','Avenue Of The Americas Holiday Season Gift Expo','UN HQ Menorah','Colored Colors Holiday Market','Holiday Lighting Ceremony','Holiday Festival','Charity Law Enforcement Christmas Motorcycle Run for Toys for Tots','Astoria Christmas Parade','Christmas Caroling with NYCO','William F Moore Christmas Tree Lighting','Festival of Lights','Senator Liu Holiday Parade','ROCKAWAY BAECH SANTA SUIT 5K','Meet Santa Paws','Christmas Carol peace in the world.','The Reason for the Season Holiday Lights Ceremony','Toy Drive','Holiday Lights Night','Coke Holiday Program','Washington Square Holiday Caroling','Annual Park Avenue Tree Lighting','Bleecker Street Holiday Block Party','festival of Lights Series','Bleecker Street Holiday Block Party','festival of Lights Series'])]
df=df.reset_index(drop=True)

<ipython-input-42-ef1c207f2cb0>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[d,'start_day']=datetime.strptime(df.at[d,'start_date'], '%Y-%m-%d').date().strftime('%A')
<ipython-input-42-ef1c207f2cb0>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[d,'end_day']=datetime.strptime(df.at[d,'end_date'], '%Y-%m-%d').date().strftime('%A')


### A. **Import Necessary Libraries**

In [43]:
#!pip install pyngrok --quiet
!pip install pyngrok==7.0.1
!pip install flask-ngrok --quiet
print("Completed!")

Completed!


### B. **Get your AuthToken from ngrok using this link-** https://dashboard.ngrok.com/get-started/your-authtoken

In [44]:
# paste your AuthToken here and execute this command
#!ngrok config add-authtoken 2YzM8tzTtyawZFXXIizYKtCOxYS_55w3iNiAs4N14tVS5ASqN #KS

!ngrok config add-authtoken 2YvDmJRG3Jjn29CHftHQCHV7PQI_VDyGPLcqhuhGHdHWkETP #My own token (Xuke)

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


###C. **Make your templates directory and add files to it.**

In [45]:
#Make the template.
%mkdir templates -p

In [46]:
%%writefile templates/home.html
<!DOCTYPE html>
<html>
<head>
<title>NYC Event Explorer</title>
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootswatch@4.5.2/dist/litera/bootstrap.min.css" integrity="sha384-enpDwFISL6M3ZGZ50Tjo8m65q06uLVnyvkFO3rsoW0UC15ATBFz3QEhr3hmxpYsn" crossorigin="anonymous">
<style>
ul {
      list-style-type: none;
      margin: 0;
      padding: 0;
      background-color: #ffffff;
      overflow: hidden;
    }

    li {
      float: left;
    }

    li a {
      display: block;
      color: black;
      text-align: right;
      padding: 14px 16px;
      text-decoration: none;
    }
img {
      float: center;
    }
.center {
            text-align: center;
        }
</style>
</head>

<body style="background-color:#F4F6F5;">
<ul>
    <li><strong><a style="font-size:200%;">NYC Event Explorer</a></strong></li>
    <li><strong><a href="../" style="font-size:120%;">Home</a></strong></li>
    <li><a href="../about" style="font-size:120%;">About</a></li>
    <li><a href="../help" style="font-size:120%;">Help</a></li>
  </ul>
<div class="center">
<h1 style="font-size:500%;">Let's start making memories</h1>
<p><strong>Your personal NYC Event Guide.
Join us in exploring New York City like never before. </strong></p>
<br>
</div>
      <div class="center">
        <img
          loading="lazy"
          srcset="https://cdn.builder.io/api/v1/image/assets/TEMP/ecbfde8b-b1be-4e06-a97f-caaee80b3417?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=100 100w, https://cdn.builder.io/api/v1/image/assets/TEMP/ecbfde8b-b1be-4e06-a97f-caaee80b3417?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=200 200w, https://cdn.builder.io/api/v1/image/assets/TEMP/ecbfde8b-b1be-4e06-a97f-caaee80b3417?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=400 400w, https://cdn.builder.io/api/v1/image/assets/TEMP/ecbfde8b-b1be-4e06-a97f-caaee80b3417?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=800 800w, https://cdn.builder.io/api/v1/image/assets/TEMP/ecbfde8b-b1be-4e06-a97f-caaee80b3417?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=1200 1200w, https://cdn.builder.io/api/v1/image/assets/TEMP/ecbfde8b-b1be-4e06-a97f-caaee80b3417?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=1600 1600w, https://cdn.builder.io/api/v1/image/assets/TEMP/ecbfde8b-b1be-4e06-a97f-caaee80b3417?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=2000 2000w, https://cdn.builder.io/api/v1/image/assets/TEMP/ecbfde8b-b1be-4e06-a97f-caaee80b3417?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&"
          class="img"
        />
      </div>
<br>
<div class="center">
<h3>Event Search</h3>
<form action="/dropdown" method="post">
  <label for="event_borough"><strong>Where are you?</strong></label><br>
  <select name="event_borough" id="event_borough">
    <option value="-Select a Borough-">-Select a Borough-</option>
    <option value="Manhattan">Manhattan</option>
    <option value="Brooklyn">Brooklyn</option>
    <option value="Queens">Queens</option>
    <option value="Bronx">Bronx</option>
    <option value="Staten Island">Staten Island</option>
  </select>
  <br><br>
  <label for="week_day"><strong>What days of the week you are interested?</strong></label><br>
  <select name="week_day" id="week_day">
    <option value="-Select Days-">-Select Days-</option>
    <option value="Weekday">Weekday</option>
    <option value="Weekend">Weekend</option>
  </select>
  <br><br>
  <input type="submit" value="Search" name='submit'>
</form>
<br><br>

<h3 class="gfg" >Login</h3>
<h3 class="strong" >{{info}} </h3>
<form action="/login" method="post">
  <h3></h3><strong>Username:</strong> <br>
  <input type="text" name='username'><br>
  <strong>Password:</strong><br>
  <input type = "password" name='password' ><br><br>
<input  type="submit" value="Login" name = 'login'>
</form>
</div>

</body>
</html>

Overwriting templates/home.html


In [47]:
%%writefile templates/about.html
<!DOCTYPE html>
<html>
<head>
<title>NYC Event Explorer</title>
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootswatch@4.5.2/dist/litera/bootstrap.min.css" integrity="sha384-enpDwFISL6M3ZGZ50Tjo8m65q06uLVnyvkFO3rsoW0UC15ATBFz3QEhr3hmxpYsn" crossorigin="anonymous">
<style>
ul {
      list-style-type: none;
      margin: 0;
      padding: 0;
      background-color: #ffffff;
      overflow: hidden;
    }

    li {
      float: left;
    }

    li a {
      display: block;
      color: black;
      text-align: right;
      padding: 14px 16px;
      text-decoration: none;
    }
 h2 {
      text-decoration: underline;
    }
.image-container {
      display: inline-block;
      margin-right: 10px;
    }
.custom-image {
      width: 300px;
      height: auto;
    }
.inline-paragraph {
      display: inline-block;
      margin-right: 220px;
    }
</style>
</head>

<body style="background-color:#F4F6F5;">
<ul>
    <li><strong><a style="font-size:200%;">NYC Event Explorer</a></strong></li>
    <li><a href="../" style="font-size:120%;">Home</a></strong></li>
    <li><strong><a href="../about" style="font-size:120%;">About</a></strong></li>
    <li><a href="../help" style="font-size:120%;">Help</a></li>
  </ul>
<br>
<h2>Our Mission</h2>
<p>In the heart of NYC, every corner brings
a new adventure, a unique experience, a story
waiting to be told. Our mission is to connect you with  these stories. Where you are local or a visitor, we believe there is an event in NYC that’s just right for you. </p>
<br>
<h2>Why Choose Us?</h2>
<p>
Tailored to your  lifestyle, we understand  that everyone’s tastes are different.  That’s why our  Editor’s Choices are as unique as you are.
</p>
<br>
<h2>Meet the Team</h2>
<p>NYC Event Explorer is brought to you by a team of NYC enthusiasts . We're a diverse group united by our love for this city and our commitment to enhancing your experience within it.</p>
<div class=".image-container">
<img
  loading="lazy"
  srcset="https://cdn.builder.io/api/v1/image/assets/TEMP/7b51bd8ea203d1874b6be8bb5043f1930c32af885efa6f38b9fd93103969b6a3?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=100 100w, https://cdn.builder.io/api/v1/image/assets/TEMP/7b51bd8ea203d1874b6be8bb5043f1930c32af885efa6f38b9fd93103969b6a3?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=200 200w, https://cdn.builder.io/api/v1/image/assets/TEMP/7b51bd8ea203d1874b6be8bb5043f1930c32af885efa6f38b9fd93103969b6a3?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=400 400w, https://cdn.builder.io/api/v1/image/assets/TEMP/7b51bd8ea203d1874b6be8bb5043f1930c32af885efa6f38b9fd93103969b6a3?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=800 800w, https://cdn.builder.io/api/v1/image/assets/TEMP/7b51bd8ea203d1874b6be8bb5043f1930c32af885efa6f38b9fd93103969b6a3?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=1200 1200w, https://cdn.builder.io/api/v1/image/assets/TEMP/7b51bd8ea203d1874b6be8bb5043f1930c32af885efa6f38b9fd93103969b6a3?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=1600 1600w, https://cdn.builder.io/api/v1/image/assets/TEMP/7b51bd8ea203d1874b6be8bb5043f1930c32af885efa6f38b9fd93103969b6a3?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=2000 2000w, https://cdn.builder.io/api/v1/image/assets/TEMP/7b51bd8ea203d1874b6be8bb5043f1930c32af885efa6f38b9fd93103969b6a3?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&"
  class="custom-image"
/>
<img
  loading="lazy"
  srcset="https://cdn.builder.io/api/v1/image/assets/TEMP/e355de788a194967dbad59c0de59628022ef6923907a753e5ab24824b9bf1da3?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=100 100w, https://cdn.builder.io/api/v1/image/assets/TEMP/e355de788a194967dbad59c0de59628022ef6923907a753e5ab24824b9bf1da3?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=200 200w, https://cdn.builder.io/api/v1/image/assets/TEMP/e355de788a194967dbad59c0de59628022ef6923907a753e5ab24824b9bf1da3?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=400 400w, https://cdn.builder.io/api/v1/image/assets/TEMP/e355de788a194967dbad59c0de59628022ef6923907a753e5ab24824b9bf1da3?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=800 800w, https://cdn.builder.io/api/v1/image/assets/TEMP/e355de788a194967dbad59c0de59628022ef6923907a753e5ab24824b9bf1da3?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=1200 1200w, https://cdn.builder.io/api/v1/image/assets/TEMP/e355de788a194967dbad59c0de59628022ef6923907a753e5ab24824b9bf1da3?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=1600 1600w, https://cdn.builder.io/api/v1/image/assets/TEMP/e355de788a194967dbad59c0de59628022ef6923907a753e5ab24824b9bf1da3?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=2000 2000w, https://cdn.builder.io/api/v1/image/assets/TEMP/e355de788a194967dbad59c0de59628022ef6923907a753e5ab24824b9bf1da3?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&"
  class="custom-image"
/>
<img
  loading="lazy"
  srcset="https://cdn.builder.io/api/v1/image/assets/TEMP/fc07bbcc25a9dec9bac8bccff50bddf50a355c1aea67d0f5ae1204cfb4ea8c2f?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=100 100w, https://cdn.builder.io/api/v1/image/assets/TEMP/fc07bbcc25a9dec9bac8bccff50bddf50a355c1aea67d0f5ae1204cfb4ea8c2f?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=200 200w, https://cdn.builder.io/api/v1/image/assets/TEMP/fc07bbcc25a9dec9bac8bccff50bddf50a355c1aea67d0f5ae1204cfb4ea8c2f?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=400 400w, https://cdn.builder.io/api/v1/image/assets/TEMP/fc07bbcc25a9dec9bac8bccff50bddf50a355c1aea67d0f5ae1204cfb4ea8c2f?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=800 800w, https://cdn.builder.io/api/v1/image/assets/TEMP/fc07bbcc25a9dec9bac8bccff50bddf50a355c1aea67d0f5ae1204cfb4ea8c2f?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=1200 1200w, https://cdn.builder.io/api/v1/image/assets/TEMP/fc07bbcc25a9dec9bac8bccff50bddf50a355c1aea67d0f5ae1204cfb4ea8c2f?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=1600 1600w, https://cdn.builder.io/api/v1/image/assets/TEMP/fc07bbcc25a9dec9bac8bccff50bddf50a355c1aea67d0f5ae1204cfb4ea8c2f?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=2000 2000w, https://cdn.builder.io/api/v1/image/assets/TEMP/fc07bbcc25a9dec9bac8bccff50bddf50a355c1aea67d0f5ae1204cfb4ea8c2f?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&"
  class="custom-image"
/>
<img
  loading="lazy"
  srcset="https://cdn.builder.io/api/v1/image/assets/TEMP/389c218b17559513da2695a95b86b8dccdadb4d023d391f99226f9628dfab831?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=100 100w, https://cdn.builder.io/api/v1/image/assets/TEMP/389c218b17559513da2695a95b86b8dccdadb4d023d391f99226f9628dfab831?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=200 200w, https://cdn.builder.io/api/v1/image/assets/TEMP/389c218b17559513da2695a95b86b8dccdadb4d023d391f99226f9628dfab831?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=400 400w, https://cdn.builder.io/api/v1/image/assets/TEMP/389c218b17559513da2695a95b86b8dccdadb4d023d391f99226f9628dfab831?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=800 800w, https://cdn.builder.io/api/v1/image/assets/TEMP/389c218b17559513da2695a95b86b8dccdadb4d023d391f99226f9628dfab831?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=1200 1200w, https://cdn.builder.io/api/v1/image/assets/TEMP/389c218b17559513da2695a95b86b8dccdadb4d023d391f99226f9628dfab831?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=1600 1600w, https://cdn.builder.io/api/v1/image/assets/TEMP/389c218b17559513da2695a95b86b8dccdadb4d023d391f99226f9628dfab831?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=2000 2000w, https://cdn.builder.io/api/v1/image/assets/TEMP/389c218b17559513da2695a95b86b8dccdadb4d023d391f99226f9628dfab831?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&"
  class="custom-image"
/>
</div>
<div class="inline-paragraph">
<p><strong>Asher Vaysblat</strong></p>
</div>
<div class="inline-paragraph">
<p><strong>Shah Alam</strong></p>
</div>
<div class="inline-paragraph">
<p><strong>Xuke Chen</strong></p>
</div>
<div class="inline-paragraph">
<p><strong>Inna Diambekova</strong></p>
</div>
</body>
</html>

Overwriting templates/about.html


In [48]:
%%writefile templates/help.html
<!DOCTYPE html>
<html>
<head>
<title>NYC Event Explorer</title>
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootswatch@4.5.2/dist/litera/bootstrap.min.css" integrity="sha384-enpDwFISL6M3ZGZ50Tjo8m65q06uLVnyvkFO3rsoW0UC15ATBFz3QEhr3hmxpYsn" crossorigin="anonymous">
<style>
ul {
      list-style-type: none;
      margin: 0;
      padding: 0;
      background-color: #ffffff;
      overflow: hidden;
    }

    li {
      float: left;
    }

    li a {
      display: block;
      color: black;
      text-align: right;
      padding: 14px 16px;
      text-decoration: none;
    }
.center {
            text-align: center;
        }
</style>
</head>

<body style="background-color:#F4F6F5;">
<ul>
    <li><strong><a style="font-size:200%;">NYC Event Explorer</a></strong></li>
    <li><a href="../" style="font-size:120%;">Home</a></strong></li>
    <li><a href="../about" style="font-size:120%;">About</a></strong></li>
    <li><strong><a href="../help" style="font-size:120%;">Help</a></strong></li>
  </ul>
<br>
<div class="center">
<h2>FAQ</h2>
<p>Q: What is NYC EXPLORER?</p>
<p>A: NYC EXPLORER is an application designed to help both locals and tourists discover and explore the many attractions of New York City, from famous landmarks to hidden gems.</p>
<br>
<p>Q: How do I find places to visit in NYC using the app?</p>
<p>A: You can search for places by selecting boroughs and weekday or weekend. </p>
<br>
<p>Q: Is there a cost to use NYC EXPLORER?</p>
<p>A: The basic version of NYC EXPLORER is free to all users. We also offer a premium version with Editor’s Choice Selection of Events</p>
<br>
<p>Q: Does the app provide real-time updates on attractions?</p>
<p>A: Yes, NYC EXPLORER offers real-time updates on various attractions, including opening hours, ticket availability, and special events.</p>
<img
  loading="lazy"
  srcset="https://cdn.builder.io/api/v1/image/assets/TEMP/309339070d9b9028da091f4d16e6927f27fb15f8f161bea48de3e0c26bd9048d?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=100 100w, https://cdn.builder.io/api/v1/image/assets/TEMP/309339070d9b9028da091f4d16e6927f27fb15f8f161bea48de3e0c26bd9048d?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=200 200w, https://cdn.builder.io/api/v1/image/assets/TEMP/309339070d9b9028da091f4d16e6927f27fb15f8f161bea48de3e0c26bd9048d?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=400 400w, https://cdn.builder.io/api/v1/image/assets/TEMP/309339070d9b9028da091f4d16e6927f27fb15f8f161bea48de3e0c26bd9048d?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=800 800w, https://cdn.builder.io/api/v1/image/assets/TEMP/309339070d9b9028da091f4d16e6927f27fb15f8f161bea48de3e0c26bd9048d?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=1200 1200w, https://cdn.builder.io/api/v1/image/assets/TEMP/309339070d9b9028da091f4d16e6927f27fb15f8f161bea48de3e0c26bd9048d?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=1600 1600w, https://cdn.builder.io/api/v1/image/assets/TEMP/309339070d9b9028da091f4d16e6927f27fb15f8f161bea48de3e0c26bd9048d?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=2000 2000w, https://cdn.builder.io/api/v1/image/assets/TEMP/309339070d9b9028da091f4d16e6927f27fb15f8f161bea48de3e0c26bd9048d?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&"
  width="500" height="500"
/>
<br><br>
<br>
<h2>Contact Us</h2>
<p>Phone: (123) 456-7890<br>Email: contact@nyceventexplorer.com</p>
<p>123 45th Street<br>New York, NY 10000</p>
</div>

</body>
</html>

Overwriting templates/help.html


In [49]:
%%writefile templates/results.html
<!DOCTYPE html>
<html>
<head>
<title>NYC Event Explorer</title>
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootswatch@4.5.2/dist/litera/bootstrap.min.css" integrity="sha384-enpDwFISL6M3ZGZ50Tjo8m65q06uLVnyvkFO3rsoW0UC15ATBFz3QEhr3hmxpYsn" crossorigin="anonymous">
<style>
ul {
      list-style-type: none;
      margin: 0;
      padding: 0;
      background-color: #ffffff;
      overflow: hidden;
    }

    li {
      float: left;
    }

    li a {
      display: block;
      color: black;
      text-align: right;
      padding: 14px 16px;
      text-decoration: none;
    }
</style>
</head>

<body style="background-color:#F4F6F5;">
<ul>
    <li><strong><a style="font-size:200%;">NYC Event Explorer</a></strong></li>
    <li><a href="../" style="font-size:120%;">Home</a></strong></li>
    <li><a href="../about" style="font-size:120%;">About</a></strong></li>
    <li><a href="../help" style="font-size:120%;">Help</a></li>
  </ul>
<br>
<h2>Search Results</h2>
<p>There are <strong>{{n}}</strong> events that match your preferences:</p>
<p>{{boro}} | {{days}}</p>
<br><br>
<p><strong>Events in next 7 Days</strong></p>
<div>
  {{f_e_7 | safe}}
</div>
<br><br>

<p><strong>Events in a Later Time:</strong></p>
<div>
  {{f_e | safe}}
</div>

</body>
</html>

Overwriting templates/results.html


In [50]:
%%writefile templates/login.html
<!DOCTYPE html>
<html>
<head>
<title>NYC Event Explorer</title>
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootswatch@4.5.2/dist/litera/bootstrap.min.css" integrity="sha384-enpDwFISL6M3ZGZ50Tjo8m65q06uLVnyvkFO3rsoW0UC15ATBFz3QEhr3hmxpYsn" crossorigin="anonymous">
<style>
ul {
      list-style-type: none;
      margin: 0;
      padding: 0;
      background-color: #ffffff;
      overflow: hidden;
    }

    li {
      float: left;
    }

    li a {
      display: block;
      color: black;
      text-align: right;
      padding: 14px 16px;
      text-decoration: none;
    }
</style>
</head>

<body style="background-color:#F4F6F5;">
<ul>
    <li><strong><a style="font-size:200%;">NYC Event Explorer</a></strong></li>
    <li><a href="../" style="font-size:120%;">Home</a></strong></li>
    <li><a href="../about" style="font-size:120%;">About</a></strong></li>
    <li><a href="../help" style="font-size:120%;">Help</a></li>
  </ul>
<br>
<h2>Editor's Choice</h2>
<img
  loading="lazy"
  srcset="https://cdn.builder.io/api/v1/image/assets/TEMP/c4c3f8d4-e61f-4f1e-958a-9d0da674e5a9?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=100 100w, https://cdn.builder.io/api/v1/image/assets/TEMP/c4c3f8d4-e61f-4f1e-958a-9d0da674e5a9?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=200 200w, https://cdn.builder.io/api/v1/image/assets/TEMP/c4c3f8d4-e61f-4f1e-958a-9d0da674e5a9?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=400 400w, https://cdn.builder.io/api/v1/image/assets/TEMP/c4c3f8d4-e61f-4f1e-958a-9d0da674e5a9?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=800 800w, https://cdn.builder.io/api/v1/image/assets/TEMP/c4c3f8d4-e61f-4f1e-958a-9d0da674e5a9?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=1200 1200w, https://cdn.builder.io/api/v1/image/assets/TEMP/c4c3f8d4-e61f-4f1e-958a-9d0da674e5a9?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=1600 1600w, https://cdn.builder.io/api/v1/image/assets/TEMP/c4c3f8d4-e61f-4f1e-958a-9d0da674e5a9?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&width=2000 2000w, https://cdn.builder.io/api/v1/image/assets/TEMP/c4c3f8d4-e61f-4f1e-958a-9d0da674e5a9?apiKey=2e880f8a63264fa0ae8a2beed9383f6d&"
/>
<br><br>
<p>Thanks for being a valued member of NYC Event Explorer! </p>
<p>Here is our editors' choice for you to explore NYC during the holiday season.</p>
<br>
<div>
  {{e_c | safe}}
</div>
</body>
</html>

Overwriting templates/login.html


### D.**Website**

In [51]:
from flask import Flask, render_template, request, session, redirect, url_for
from flask_ngrok import run_with_ngrok
from datetime import datetime, date, timedelta
import pytz # needed to set time zone
import pandas as pd
#from flask_session import Session


app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is running
app.secret_key = 'FundamentalsOfPython'

database = {'NYC': 'nyc'}

#Get future events after 7 days method
def get_events(event_borough, week_day):
    future_events=pd.DataFrame()
    date = datetime.now(pytz.timezone("US/Eastern")).date()
    date_7 = date + timedelta(days=7)
    for x in df.index:
      if datetime.strptime(df.at[x,'start_date'], '%Y-%m-%d').date() > date_7:
        future_events = future_events.append(df.iloc[x])
    future_events['start_date'] = pd.to_datetime(df['start_date'])#converting string to date
    future_events=future_events.sort_values(by='start_date')#sort events chronologically
    if event_borough != '-Select a Borough-':
      future_events=future_events[future_events['event_borough']==event_borough]
    if week_day != '-Select Days-':
      if week_day == "Weekday":
        future_events=future_events[future_events['start_day'].isin(["Monday","Tuesday","Wednesday","Thursday","Friday"])]
      if week_day == "Weekend":
        future_events=future_events[future_events['start_day'].isin(["Saturday","Sunday"])]
    f_e=future_events.to_html(index=False)
    return f_e

#Get future 7 days events method
def get_events_7(event_borough, week_day):
    future_events_7=pd.DataFrame()
    date = datetime.now(pytz.timezone("US/Eastern")).date()
    date_7 = date + timedelta(days=7)
    for x in df.index:
      if date <= datetime.strptime(df.at[x,'start_date'], '%Y-%m-%d').date() <= date_7:
        future_events_7 = future_events_7.append(df.iloc[x])
    future_events_7['start_date'] = pd.to_datetime(df['start_date'])#converting string to date
    future_events_7=future_events_7.sort_values(by='start_date')#sort events chronologically
    if event_borough != '-Select a Borough-':
      future_events_7=future_events_7[future_events_7['event_borough']==event_borough]
    if week_day != '-Select Days-':
      if week_day == "Weekday":
        future_events_7=future_events_7[future_events_7['start_day'].isin(["Monday","Tuesday","Wednesday","Thursday","Friday"])]
      if week_day == "Weekend":
        future_events_7=future_events_7[future_events_7['start_day'].isin(["Saturday","Sunday"])]
    f_e_7=future_events_7.to_html(index=False)
    return f_e_7

#Get editor's choice method
def get_e_c():
    editor_choice = pd.DataFrame()
    current_date = datetime.now(pytz.timezone("US/Eastern")).date()
    date_2024=date(2024, 1, 1)
    for index, row in editor.iterrows():
        start_date = datetime.strptime(row['start_date'], '%Y-%m-%d').date()

        if current_date <= start_date <= date_2024:
            editor_choice = editor_choice.append(row)

    editor_choice['start_date'] = pd.to_datetime(editor_choice['start_date'])
    editor_choice = editor_choice.sort_values(by='start_date')
    e_c = editor_choice.to_html(index=False)
    return e_c

#Home page
@app.route("/")
def home():
  return render_template('home.html')

@app.route('/dropdown', methods=['POST', 'GET'])
def dropdown():
    if request.method=='POST':
        event_borough = request.form['event_borough']
        week_day = request.form['week_day']
        if event_borough != "-Select a Borough-":
          boro=event_borough
        else:
          boro="All Boroughs"
        if week_day != "-Select Days-":
          days=week_day
        else:
          days="All Days"
        f_e=get_events(event_borough, week_day)
        f_e_7=get_events_7(event_borough, week_day)
        n1=len(pd.read_html(f_e_7)[0])
        n2=len(pd.read_html(f_e)[0])
        n=n1+n2
        return render_template('results.html', f_e=f_e, n=n, f_e_7=f_e_7, boro=boro, days=days)

@app.route('/login', methods = ['POST', 'GET'])
def login():
    if request.method == "POST":
      username = request.form['username']
      password = request.form['password']
      login = request.form.get('login')
      if login is not None:
      # Non registered user
        if  username not in database:
          return render_template('home.html',
                                  info='Invalid User ????!')
        else:
          # Incorrect password
          if database[username] != password:
            return render_template('home.html',
                                    info='Invalid Password ????!')
          else:
            session['username'] = request.form['username'] # Store the username in a session if logged in correctly
            e_c=get_e_c()
            return render_template('login.html', e_c=e_c)
    else:
      return render_template('home.html')


#About page
@app.route("/about")
def about():
  return render_template("about.html")

#Help page
@app.route("/help")
def help():
  return render_template("help.html")

In [52]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://69f5-34-82-230-166.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [18/Jan/2024 00:08:42] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2024 00:08:43] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [18/Jan/2024 00:09:04] "GET / HTTP/1.1" 200 -
